In [1]:
%matplotlib inline
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar_template as lct
import matplotlib.pyplot as plt

In [2]:
training_features_1,training_labels_1 = pickle.load(open('preprocess_batch_1.p','rb'))
training_features_2,training_labels_2 = pickle.load(open('preprocess_batch_2.p','rb'))
training_features_3,training_labels_3 = pickle.load(open('preprocess_batch_3.p','rb'))
training_features_4,training_labels_4 = pickle.load(open('preprocess_batch_4.p','rb'))
training_features_5,training_labels_5 = pickle.load(open('preprocess_batch_5.p','rb'))

valid_features, valid_labels = pickle.load(open('preprocess_validation.p','rb'))
test_features, test_labels = pickle.load(open('preprocess_testing.p','rb'))

<h1>Define Hyper-perparmeter</h1>

In [3]:
#learning rate
lr = 0.0001
#number of epochs
epochs = 10
#number of batch_size
batch_size = 64

#network parameters
n_hidden_1 = 2048
n_hidden_2 = 1024
n_hidden_3 = 512
num_input = training_features_1[0, :].shape[0] * training_features_1[0, :].shape[1] * training_features_1[0, :].shape[2]
num_classes = 10

In [4]:
def reshape_1d(training_features):
    
    features_reshaped = []

    for image in training_features:
        
        arr = np.array([])

        r = image[:, :, 0].reshape(1024)
        g = image[:, :, 1].reshape(1024)
        b = image[:, :, 2].reshape(1024)

        arr = np.concatenate((r,g,b))
        
        features_reshaped.append(arr)

    return np.stack(features_reshaped)

In [5]:
train_1 = reshape_1d(training_features_1)
train_2 = reshape_1d(training_features_2)
train_3 = reshape_1d(training_features_3)
train_4 = reshape_1d(training_features_4)
train_5 = reshape_1d(training_features_5)

train_features = np.vstack([train_1, train_2, train_3, train_4, train_5])
train_labels = np.vstack([training_labels_1, training_labels_2, training_labels_3, training_labels_4, training_labels_5])

In [6]:
test_features = reshape_1d(test_features)
valid_features = reshape_1d(valid_features)

In [7]:
tf.reset_default_graph()

<h1>Define Placeholder</h1>

In [8]:
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

initializer = tf.contrib.layers.variance_scaling_initializer()

#Layers weight & bias
weights = {
    'W1': tf.Variable(initializer([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(initializer([n_hidden_1, n_hidden_2]),name='W2'),
    'W3': tf.Variable(initializer([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(initializer([n_hidden_3, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


<h1>Define Neural Network Architecture</h1>

In [9]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.nn.elu(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
    layer_2_out = tf.nn.elu(tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']))
    layer_3_out = tf.nn.elu(tf.add(tf.matmul(layer_2_out,weights['W3']),biases['b3']))
    out = tf.add(tf.matmul(layer_3_out,weights['Wout']),biases['bout'])
    return out

<h1>Define cost andoptimization</h1>

In [10]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [11]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(epochs):
        #fetch batch
        for batch_x, batch_y in lct.mini_batch(train_features, train_labels, batch_size):
    
            sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
            
        acc_train = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
        acc_valid = sess.run(accuracy,feed_dict={X:valid_features, Y:valid_labels})
        L = sess.run(loss,feed_dict={X:batch_x, Y:batch_y})
        print("epoch "+str(i)+", Accuracy_train= {:.3f}".format(acc_train))
        print("epoch "+str(i)+", Accuracy_valid= {:.3f}".format(acc_valid))
        print(L)
    
    print("Training finished!")
    
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels}))

100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:13<00:00, 51.16it/s]


epoch 0, Accuracy_train= 0.297
epoch 0, Accuracy_valid= 0.398
1.8087828


100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:13<00:00, 52.23it/s]


epoch 1, Accuracy_train= 0.406
epoch 1, Accuracy_valid= 0.440
1.668611


100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:13<00:00, 51.38it/s]


...

epoch 7, Accuracy_train= 0.688
epoch 7, Accuracy_valid= 0.499
1.0557438


100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:13<00:00, 52.61it/s]


epoch 8, Accuracy_train= 0.750
epoch 8, Accuracy_valid= 0.505
0.96015406


100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:13<00:00, 52.65it/s]


epoch 9, Accuracy_train= 0.781
epoch 9, Accuracy_valid= 0.505
0.8844078
Training finished!
Testing Accuracy: 0.5052
